In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


### Bistability

In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [3]:
##### LOAD BOUNDARIES
data_file = 'bi.pickle'
with open(data_file,'rb') as f:
    load_array= pickle.load(f)
exc = load_array[0]
inh = load_array[1]
print(len(exc))

147


In [4]:
t0_array = np.arange(380., -1, -1.)
t_pen = t0_array / 400.

bestControl_1 = [None] * len(t_pen)
bestState_1 = [None] * len(t_pen)
cost_1 = [None] * len(t_pen)
runtime_1 = [None] * len(t_pen)
grad_1 = [None] * len(t_pen)
phi_1 = [None] * len(t_pen)
costnode_1 = [None] * len(t_pen)
weights_1 = [None] * len(t_pen)

In [5]:
initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

cgv_list = [None, "HS", "FR", "PR", "CD", "LS", "DY", "WYL", "HZ", None]

In [6]:
dur_pre = 10
dur_post = 10

i = 20

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-32
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

c_var = [ [0], [1], [0,1]]
p_var = [ [0], [0], [0]]

### CURRENTS
cntrl_vars_0 = [0,1]
prec_vars = [0]

max_I = [3., -3.]
factor_wp = 1. / 0.05
factor_ws = 1.
factor_we = 0.
dur = 400
trans_time = 0.95
    
maxC = [5., -5., 0.18, 0.]

n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [7]:
init_file = 'control_1.pickle'
final_file_1 = 'control_tradeoff_1a.pickle'

In [8]:
if os.path.isfile(init_file) :
    print("init file found")
    
    with open(init_file,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1[0] = load_array[0][i]
    bestState_1[0] = load_array[1][i]
    cost_1 [0]= load_array[2][i]
    runtime_1[0] = load_array[3][i]
    grad_1 [0]= load_array[4][i]
    phi_1[0] = load_array[5][i]
    costnode_1[0] = load_array[6][i]
    weights_1[0] = load_array[7][i]

init file found
147


In [9]:
if os.path.isfile(final_file_1) :
    print("final file found")
    
    with open(final_file_1,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1 = load_array[0]
    bestState_1 = load_array[1]
    cost_1 = load_array[2]
    runtime_1 = load_array[3]
    grad_1 = load_array[4]
    phi_1 = load_array[5]
    costnode_1 = load_array[6]
    weights_1 = load_array[7]

final file found
381


In [10]:
# get initial parameters and target states

print("------- ", i, exc[i], inh[i])
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.

aln.params.duration = 3000.

control0 = aln.getZeroControl()
control0 = functions.step_control(aln, maxI_ = max_I[0])

aln.run(control=control0)

target_rates = np.zeros((2))
target_rates[0] = aln.rates_exc[0,-1] 
target_rates[1] = aln.rates_inh[0,-1]

control0 = functions.step_control(aln, maxI_ = max_I[1])
aln.run(control=control0)

init_state_vars = np.zeros(( len(state_vars) ))
for j in range(len(state_vars)):
    if aln.state[state_vars[j]].size == 1:
        init_state_vars[j] = aln.state[state_vars[j]][0]
    else:
        init_state_vars[j] = aln.state[state_vars[j]][0,-1]

initVars[i] = init_state_vars

aln.params.duration = dur

target[i] = aln.getZeroTarget()
target[i][:,0,:] = target_rates[0]
target[i][:,1,:] = target_rates[1]

-------  20 0.4500000000000001 0.4750000000000002


In [11]:
# get uncontrolled cost

data.set_parameters(aln)
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.
aln.params.duration = dur

cgv = None

In [12]:
print(np.argmax(np.abs(bestControl_1[0]), axis=2))
print(bestControl_1[0][0,1,1650:1700]/5.)
print(bestControl_1[0][0,1,1674]/5.)

max_percent = (np.argmax(np.abs(bestControl_1[0]), axis=2)[0,1] -100) / 3800
print(max_percent)

[[1729 1674    0    0    0    0]]
[-0.1633104  -0.16379484 -0.16425582 -0.16469311 -0.16509954 -0.16548749
 -0.16585186 -0.16619199 -0.16650851 -0.16680346 -0.16707971 -0.16733875
 -0.1675742  -0.16778295 -0.16795832 -0.16819977 -0.16841262 -0.16860169
 -0.16876577 -0.16890939 -0.1690244  -0.16912678 -0.16920045 -0.16924691
 -0.16926188 -0.16925026 -0.16921142 -0.16914676 -0.16905532 -0.16894278
 -0.16879886 -0.16862746 -0.1684289  -0.16820321 -0.16795549 -0.16767893
 -0.16737551 -0.16704399 -0.16668665 -0.16630856 -0.16590178 -0.16546897
 -0.16501036 -0.16452214 -0.16400294 -0.16345805 -0.16288765 -0.16229229
 -0.1616851  -0.16105491]
-0.16926188242134096
0.4142105263157895


In [29]:
##### initial guess
weight_ = 1.
cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

setinit(initVars[i], aln)

weights_1[0] = cost.getParams()

for j in range(1):

    dur = 400. - np.around(j/10., 1)
    T = 4001 - j
    aln.params.duration = dur

    #factor_wp = dur / 20.
    factor_wp = 1.
    cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

    print('-----', j)
    setinit(initVars[i], aln)

    max_it = 10.
    if j > 170:
        max_it = 100.
    elif j > 140:
        max_it = 30.
    elif j > 100.:
        max_it = 20.
    
    if type(bestControl_1[j]) == type(None):
        total_shift = round(j * max_percent * 10)
        prev_shift = round( (j-1) * max_percent * 10)
        shift_ind = total_shift - prev_shift
        #print(total_shift, prev_shift, shift_ind)
        control0 = np.zeros(( bestControl_1[j-1][:,:,100:-101].shape ))
        control0[:,:,:-shift_ind] = 1. * bestControl_1[j-1][:,:,100+shift_ind:-101]
    else:
        if bestControl_1[j].shape[2] == T+200:
            control0 = bestControl_1[j][:,:,100:-100]
        else:
            control0 = bestControl_1[j][:,:,100:-(100+j)]

    bestControl_1[j], bestState_1[j], cost_1[j], runtime_1[j], grad_1[j], phi_1[j], costnode_1[j] = aln.A1(
        control0, target[i], c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = [0,1],
        prec_variables_ = prec_vars, transition_time_ = t_pen[j])

    print(costnode_1[j][0][0][0], costnode_1[j][2][0][1])

    print(costnode_1[0][0][0][0], costnode_1[0][2][0][1])
        
    with open(final_file_1,'wb') as f:
        pickle.dump([bestControl_1, bestState_1, cost_1, runtime_1, grad_1, phi_1,
                costnode_1, weights_1], f)

set cost params:  1.0 0.0 1.0
set cost params:  1.0 0.0 1.0
----- 368
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  8.07663369348403
Gradient descend method:  None
RUN  1 , total integrated cost =  8.076434545104526
RUN  2 , total integrated cost =  8.07624614761409
RUN  3 , total integrated cost =  8.07603202720242
RUN  4 , total integrated cost =  8.075831469639974
RUN  5 , total integrated cost =  8.07562986786866
RUN  6 , total integrated cost =  8.075420198882432
RUN  7 , total integrated cost =  8.075222046575119
RUN  8 , total integrated cost =  8.07503340234114
RUN  9 , total integrated cost =  8.074820012990523
RUN  10 , total integrated cost =  8.074619401807267
RUN  11 , total integrated cost =  8.074418440144607
RUN  12 , total integrated cost =  8.07420867484301
RUN  13 , total integrated cost =  8.074011170571419
RUN  14 , total integrated cost =  8.07382221941221
RUN  15 , total integrated cost =  8.073609237390365
RUN  16 , total integrated cos

RUN  12 , total integrated cost =  10.014752879604242
RUN  13 , total integrated cost =  10.014452401696147
RUN  14 , total integrated cost =  10.014138611296499
RUN  15 , total integrated cost =  10.013838186808655
RUN  16 , total integrated cost =  10.013524491531173
RUN  17 , total integrated cost =  10.01322416745013
RUN  18 , total integrated cost =  10.012910559845821
RUN  19 , total integrated cost =  10.012610880572065
RUN  20 , total integrated cost =  10.012297235161
RUN  30 , total integrated cost =  10.009231148481142
RUN  40 , total integrated cost =  10.006170034397366
RUN  50 , total integrated cost =  10.00311316140527
RUN  60 , total integrated cost =  10.000061583898622
RUN  70 , total integrated cost =  9.997014128579286
RUN  80 , total integrated cost =  9.99328219243809
RUN  90 , total integrated cost =  9.990351041179443
RUN  100 , total integrated cost =  9.985750570207287
RUN  100 , total integrated cost =  9.985750570207287
Improved over  100  iterations in  6.

RUN  60 , total integrated cost =  14.949328269895913
RUN  70 , total integrated cost =  14.941448850131936
RUN  80 , total integrated cost =  14.933863166216309
RUN  90 , total integrated cost =  14.92637957095823
RUN  100 , total integrated cost =  14.918938146852636
RUN  100 , total integrated cost =  14.918938146852636
Improved over  100  iterations in  6.978580254999542  seconds by  0.4951494843429032  percent.
1.893406036931207 2.8637686921495638
5.7868700186056746e-08 3.3312300207483534
set cost params:  1.0 0.0 1.0
----- 377
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  296.3336139887507
Gradient descend method:  None
RUN  1 , total integrated cost =  296.33328333187336
RUN  2 , total integrated cost =  296.33295747520333
RUN  3 , total integrated cost =  296.3325648107732
RUN  4 , total integrated cost =  296.33220833242865
RUN  5 , total integrated cost =  296.3318857545078
RUN  6 , total integrated cost =  296.3315932926734
RUN  7 , total integrated

In [78]:
final_file_2 = 'control_tradeoff_1a_2.pickle'
cost_node_array = np.zeros(( len(t_pen),3 )) # S i, S e, P

bestControl_2 = [None] * len(t_pen)
bestState_2 = [None] * len(t_pen)
cost_2 = [None] * len(t_pen)
runtime_2 = [None] * len(t_pen)
grad_2 = [None] * len(t_pen)
phi_2 = [None] * len(t_pen)
costnode_2 = [None] * len(t_pen)
weights_2 = [None] * len(t_pen)

bestControl_2[0], bestState_2[0], cost_2[0], runtime_2[0], grad_2[0], phi_2[0], costnode_2[0], weights_2[0] = bestControl_1[0], bestState_1[0], cost_1[0], runtime_1[0], grad_1[0], phi_1[0], costnode_1[0], weights_1[0]



In [ ]:
if os.path.isfile(final_file_2) :
    print("final file 2 found")
    
    with open(final_file_2,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_2 = load_array[0]
    bestState_2 = load_array[1]
    cost_2 = load_array[2]
    runtime_2 = load_array[3]
    grad_2 = load_array[4]
    phi_2 = load_array[5]
    costnode_2 = load_array[6]
    weights_2 = load_array[7]

    cost_node_array[0] = [costnode_1[0][2][0][1], costnode_1[0][2][0][0], costnode_1[0][0][0][0]]

    for j in range(1, len(costnode_2)):
        if type(costnode_2[j]) != type(None):
            cost_node_array[j] = [costnode_2[j][2][0][1], costnode_2[j][2][0][0], costnode_2[j][0][0][0]]

In [82]:
##### initial guess
weight_ = 1.
w_p, w_s, w_e = 1., 1., 0.
cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

maxcost_s = 1.01 * ( costnode_1[0][2][0][0] + costnode_1[0][2][0][1] )
maxcost_p = 1e10 * costnode_1[0][0][0][0]

w_s = weights_2[269][2]

for j in range(270, 281):

    dur = 400. - np.around(j/10., 1)
    T = 4001 - j
    aln.params.duration = dur

    w_p = dur / 20.
    cost.setParams(w_p, w_e, w_s)

    print('-----', j)
    setinit(initVars[i], aln)

    max_it = 10.
    
    if type(bestControl_2[j]) == type(None):
        if w_s > 1.:
            total_shift = round(j * max_percent * 10)
            prev_shift = round( (j-1) * max_percent * 10)
            shift_ind = total_shift - prev_shift
            #print(total_shift, prev_shift, shift_ind)
            control0 = np.zeros(( bestControl_2[j-1][:,:,100:-101].shape ))
            control0[:,:,:] = 1. * bestControl_2[j-1][:,:,100+shift_ind:-101+shift_ind]
        elif w_s == 1.:
            control0 = np.zeros(( bestControl_1[j][:,:,100:-100].shape ))
            control0[:,:,:] = 1. * bestControl_1[j][:,:,100:-100]
    else:
        cost.setParams(weights_2[j][0], weights_2[j][1], weights_2[j][2])
        if bestControl_2[j].shape[2] == T+200:
            control0 = bestControl_2[j][:,:,100:-100]
        else:
            control0 = bestControl_2[j][:,:,100:-(100+j)]

    bestControl_2[j], bestState_2[j], cost_2[j], runtime_2[j], grad_2[j], phi_2[j], costnode_2[j] = aln.A1(
        control0, target[i], c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = [0,1],
        prec_variables_ = prec_vars, transition_time_ = t_pen[j])
        
    sumcost_s =  (costnode_2[j][2][0][0] + costnode_2[j][2][0][1])
    cost_p = costnode_2[j][0][0][0]

    print("costs = ", sumcost_s, cost_p)
    print('cost limits = ', maxcost_s, maxcost_p)

    if cost_p > maxcost_p:
        print("beyond max precision cost, break")
        break
    elif sumcost_s < maxcost_s:
        print("within sparsity cost limit, continue")
        cost_node_array[j] = [costnode_2[j][2][0][1], costnode_2[j][2][0][0], cost_p]
        weights_2[j] = cost.getParams()
        with open(final_file_2,'wb') as f:
            pickle.dump([bestControl_2, bestState_2, cost_2, runtime_2, grad_2, phi_2,
                costnode_2, weights_2], f)
        continue

    counter = 0
    while sumcost_s >= maxcost_s:
        print("iteration ", counter)
        counter += 1
        print("increase sparsity weight at t_ = ", t0_array[j])
        
        w_s = round(w_s * 1.5)

        max_it = 60

        cost.setParams(w_p, w_e, w_s)
        setinit(initVars[i], aln)
        control0[:,:,:] = 1. * bestControl_2[j][:,:,100:-100]

        bestControl_2[j], bestState_2[j], cost_2[j], runtime_2[j], grad_2[j], phi_2[j], costnode_2[j] = aln.A1(
        control0, target[i], c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = [0,1],
        prec_variables_ = prec_vars, transition_time_ = t_pen[j])

        sumcost_s =  (costnode_2[j][2][0][0] + costnode_2[j][2][0][1])
        cost_p = costnode_2[j][0][0][0]

        print("costs = ", sumcost_s, cost_p)
        print('cost limits = ', maxcost_s, maxcost_p)

    cost_node_array[j] = [costnode_2[j][2][0][1], costnode_2[j][2][0][0], cost_p]
    weights_2[j] = cost.getParams()

    with open(final_file_2,'wb') as f:
        pickle.dump([bestControl_2, bestState_2, cost_2, runtime_2, grad_2, phi_2,
                costnode_2, weights_2], f)

set cost params:  18.605 0.0 1.0
set cost params:  18.65 0.0 13986
----- 270
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  47108.903084299935
Gradient descend method:  None
RUN  1 , total integrated cost =  47105.09525190344
RUN  2 , total integrated cost =  47105.05271821566
RUN  3 , total integrated cost =  47105.00168071528
RUN  4 , total integrated cost =  47104.936035252824
RUN  5 , total integrated cost =  47104.871870919815
RUN  6 , total integrated cost =  47104.832763630446
RUN  7 , total integrated cost =  47104.788899604006
RUN  8 , total integrated cost =  47104.76231458074
RUN  9 , total integrated cost =  47104.73126965594
RUN  10 , total integrated cost =  47104.70976996257
RUN  10 , total integrated cost =  47104.70976996257
Improved over  10  iterations in  8.286527188000036  seconds by  0.008901320266076596  percent.
costs =  3.3626848715312883 3.9785070630546726
cost limits =  3.364542320955837 578.6870018605674
within sparsity cost limit, c